# Initial data

In [1]:
import requests, json, datetime, pandas as pd

# Данные тестовой среды для подключения
token_url = "http://mosrepora02.evropolis.ru:8080/ords/wks01/oauth/token"
test_api_url = "http://mosrepora02.evropolis.ru:8080/ords/wks01/api/"

#Список данных для Oauth2
users = {
    'che': {
            'id': '-ZZ0ru0pMDfTNg-DHXI3VQ..',
            'secret': 'hebPXNQjEW5YDefvjc2J9A..'
        },
    'banki': {
            'id': 'Wute9o2Y7k24GBF0-L-ABg..',
            'secret': 'ZovEIGLI5-iBKqT9rgfuRQ..'
        }    
    }

#users['che']['id']

# Данные пользователя для создания
def NewSubject():
    Subject = {
                "lastname": "Тестов",
                "firstname": "Тест",
                "parentname": "Тестович",
                "sex": "М",
                "birthday": "1983-05-24T00:00:00",
                "latinname": "Gridnev Roman",
                "drivingdatebeg": "2004-01-01T00:00:00",
                "citizenship": "RUS",
                "address": [
                    {
                        "code": 2247,
                        "code_desc": "Регистрация",
                        "text": "Московская обл, Мытищи г, Воровского ул, д. 5 кв 21",
                        "street_kladr_id": "50000044000005900"
                    }
                ],
                "contact": [
                    {
                        "code": 2243,
                        "code_desc": "E-mail личный",
                        "text": "test@ya.ru"
                    },
                                    {
                        "code": 2240,
                        "text": "79264494860"
                    },

                ],
                "document": [

                    {
                        "code": 1165,
                        "code_desc": "Паспорт гражданина России",
                         "series": "1111",
                         "number": "111111",
                        "issue_date": "2003-08-12T00:00:00",
                        "issue_by": "ОВД ОВД ОВД"
                    }
               ]
            }
    return Subject

# Main functions and classes

In [2]:
# Функция получения токена Oauth 2
def GetToken(user):
    data = {'grant_type': 'client_credentials'}
    client_id = users[user]['id']
    client_secret = users[user]['secret']
    access_token_response = requests.post(token_url, data=data, verify=False, allow_redirects=False, auth=(client_id, client_secret))
    tokens = json.loads(access_token_response.text)
    return tokens['access_token']

class apirequest():
    """docstring"""
    def __init__(self, user):
        self.token = GetToken(user)
        self.api_call_headers = {'Authorization': 'Bearer ' + self.token}
        
    def getsubject(self, subjisn):
        resp = []
        timebeg = datetime.datetime.now()
        api_call_response = requests.get(test_api_url + 'subject/' + str(subjisn), headers=self.api_call_headers, verify=False)
        subject = json.loads(api_call_response.text)
        timeend = datetime.datetime.now()
        
        resp.append(api_call_response.status_code)
        resp.append((timeend - timebeg).microseconds/1000)
        resp.append(json.loads(api_call_response.text))
        return resp
    
    def newsubject(self, subject):
        resp = []
        timebeg = datetime.datetime.now()
        api_call_response = requests.post(test_api_url + 'subject', json = subject, headers=self.api_call_headers, verify=False)
        subject = json.loads(api_call_response.text)
        timeend = datetime.datetime.now()
        
        resp.append(api_call_response.status_code)
        resp.append((timeend - timebeg).microseconds/1000)
        resp.append(json.loads(api_call_response.text))
        return resp

In [7]:
api = apirequest('che')
api.newsubject(NewSubject)
#api.getsubject(446014414)
#NewSubject

[200,
 991.677,
 {'status': {'code': 'success', 'message': None},
  'result': {'data': {'id': 446014422}}}]

# Getsubject load test 

In [48]:
test_users = pd.read_csv('test_users.csv')

In [130]:
results = pd.DataFrame(columns = ['user', 
                                  'resp_time_ms',
                                  'resp_status_code',
                                  'resp_status_name',
                                  'resp_status_error',
                                  'subject_isn'])
results

,user,resp_time_ms,resp_status_code,resp_status_name,resp_status_error,subject_isn


In [133]:
n=0
timebeg = datetime.datetime.now()
for user in users.keys():
    api = apirequest(user)
    for isn in test_users:
        resp = api.getsubject(isn)
        results.loc[n] = {'user': user, 
                          'resp_time_ms': resp[1], 
                          'resp_status_code': resp[0], 
                          'resp_status_name': resp[2]['status']['code'], 
                          'resp_status_error': resp[2]['status']['message'], 
                          'subject_isn': isn}
        n += 1
timeend = datetime.datetime.now()
print('Время, затраченное на тест:', timeend-timebeg)       
    

время, затраченное на тест: 0:00:07.736359


In [132]:
results

,user,resp_time_ms,resp_status_code,resp_status_name,resp_status_error,subject_isn
0,che,660.227,200,success,None,445903407
1,che,442.125,200,success,None,445903412
2,che,358.076,200,success,None,445903414
3,banki,417.876,200,success,None,445903407
4,banki,411.895,200,success,None,445903412
5,banki,512.661,200,success,None,445903414


In [127]:
results.agg({'resp_time_ms':['count', 'min', 'max', 'median', 'mean']})

,resp_time_ms
count,6.000000
min,85.049000
max,933.422000
median,117.948500
mean,352.150833


# Newsubject functionality test

In [ ]:
api = apirequest('che')
timebeg = datetime.datetime.now()
row = 1

result = pd.DataFrame(columns = ['case', 
                                  'timeelapsed',
                                 'resp_html_code',
                                  'resp_status_code',
                                  'resp_status_error',
                                  'created_subjisn',
                                  'test_result'
                                ])

def writeresult(row, resp, created_subjisn, test_result):
    result.loc[row] = {'case': case, 
                        'timeelapsed': resp[1], 
                        'resp_html_code': resp[0], 
                        'resp_status_code': resp[2]['status']['code'], 
                        'resp_status_error': resp[2]['status']['message'],
                        'created_subjisn': created_subjisn,
                        'test_result': test_result
                       }




case = 'Минимальный набор данных'
print (case)
testsubject = NewSubject()
del testsubject['parentname']
del testsubject['sex']
del testsubject['latinname']
del testsubject['drivingdatebeg']
del testsubject['citizenship']
del testsubject['address']
del testsubject['contact']
del testsubject['document']
resp = api.newsubject(testsubject)

# обрабатываем случай, когда результат не вернулся
if resp[2].get('result') == None:
    created_subjisn = None
    test_result = 'Fail'
else:
    created_subjisn = resp[2]['result']['data']['id']
    test_result = 'Ok'
  
writeresult(row, resp, created_subjisn, test_result)

case = 'Два паспорта'
print (case)
testsubject = NewSubject()
testsubject['document'] = testsubject['document']*2
resp = api.newsubject(testsubject)
# обрабатываем случай, когда результат не вернулся
if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
    created_subjisn = None
    test_result = 'Ok'
else:
    created_subjisn = resp[2]['result']['data']['id']
    test_result = 'Fail'
  
writeresult(row, resp, created_subjisn, test_result)
row += 1

case = 'Два адреса'
print (case)
testsubject = NewSubject()
testsubject['address'] = testsubject['address']*2
resp = api.newsubject(testsubject)
# обрабатываем случай, когда результат не вернулся
if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
    created_subjisn = None
    test_result = 'Ok'
else:
    created_subjisn = resp[2]['result']['data']['id']
    test_result = 'Fail'
  
writeresult(row, resp, created_subjisn, test_result)
row += 1

case = 'Документы из прошлого'
print (case)
testsubject = NewSubject()
testsubject['document'][0]['issue_date'] = '1883-05-24'
resp = api.newsubject(testsubject)
# обрабатываем случай, когда результат не вернулся
if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
    created_subjisn = None
    test_result = 'Ok'
else:
    created_subjisn = resp[2]['result']['data']['id']
    test_result = 'Fail'
writeresult(row, resp, created_subjisn, test_result)
row += 1
    
case = 'Документы из будущего'
print (case)
testsubject = NewSubject()
testsubject['document'][0]['issue_date'] = '2883-05-24'
resp = api.newsubject(testsubject)
# обрабатываем случай, когда результат не вернулся
if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
    created_subjisn = None
    test_result = 'Ok'
else:
    created_subjisn = resp[2]['result']['data']['id']
    test_result = 'Fail'    
writeresult(row, resp, created_subjisn, test_result)
row += 1

case = 'Паспорт выдан до 14 лет'
print (case)
testsubject = NewSubject()
testsubject['document'][0]['issue_date'] = '1984-05-24'
resp = api.newsubject(testsubject)
# обрабатываем случай, когда результат не вернулся
if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
    created_subjisn = None
    test_result = 'Ok'
else:
    created_subjisn = resp[2]['result']['data']['id']
    test_result = 'Fail'    
writeresult(row, resp, created_subjisn, test_result)
row += 1

case = 'Телефон не нужного формата'
print (case)
testsubject = NewSubject()
testsubject['contact'][1]['text'] = '1984-05-24'
resp = api.newsubject(testsubject)
# обрабатываем случай, когда результат не вернулся
if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
    created_subjisn = None
    test_result = 'Ok'
else:
    created_subjisn = resp[2]['result']['data']['id']
    test_result = 'Fail'    
writeresult(row, resp, created_subjisn, test_result)
row += 1

case = 'Паспорт без серии и номера'
print (case)
testsubject = NewSubject()
del testsubject['document'][0]['series']
del testsubject['document'][0]['number']
resp = api.newsubject(testsubject)
# обрабатываем случай, когда результат не вернулся
if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
    created_subjisn = None
    test_result = 'Ok'
else:
    created_subjisn = resp[2]['result']['data']['id']
    test_result = 'Fail'    
writeresult(row, resp, created_subjisn, test_result)
row += 1

case = 'Паспорт только серия и номер'
print (case)
testsubject = NewSubject()
del testsubject['document'][0]['code_desc']
del testsubject['document'][0]['issue_date']
del testsubject['document'][0]['issue_by']
resp = api.newsubject(testsubject)
# обрабатываем случай, когда результат не вернулся
if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
    created_subjisn = None
    test_result = 'Fail'
else:
    created_subjisn = resp[2]['result']['data']['id']
    test_result = 'Ok'    
writeresult(row, resp, created_subjisn, test_result)
row += 1

case = 'Попытка создания дубля'
print (case)
testsubject = NewSubject()
resp = api.newsubject(testsubject)
# обрабатываем случай, когда результат не вернулся
if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
    created_subjisn = None
    test_result = 'Fail'
else:
    created_subjisn = resp[2]['result']['data']['id']
    print('----------')
    print(testsubject)
    print('Новый контрагент: ', created_subjisn)
# Проверяем кейс с дублем по номеру телефона
    del testsubject['sex']
    del testsubject['latinname']
    del testsubject['drivingdatebeg']
    del testsubject['citizenship']
    del testsubject['address']
    del testsubject['document']
    resp2 = api.newsubject(testsubject)
    created_subjisn2 = resp2[2]['result']['data']['id']
    print('----------')
    print(testsubject)
    print('Новый контрагент: ', created_subjisn2)
    testsubject = NewSubject()
# Проверяем кейс с дублем по номеру паспорта
    del testsubject['sex']
    del testsubject['latinname']
    del testsubject['drivingdatebeg']
    del testsubject['citizenship']
    del testsubject['address']
    del testsubject['contact']
    resp3 = api.newsubject(testsubject)
    created_subjisn3 = resp3[2]['result']['data']['id']
    print('----------')
    print(testsubject)
    print('Новый контрагент: ', created_subjisn3)
    testsubject = NewSubject()
# Проверяем кейс с дублем по ФИО и дате рождения
    del testsubject['sex']
    del testsubject['latinname']
    del testsubject['drivingdatebeg']
    del testsubject['citizenship']
    del testsubject['address']
    del testsubject['contact']
    del testsubject['document']
    resp4 = api.newsubject(testsubject)
    created_subjisn4 = resp4[2]['result']['data']['id']
    print('----------')
    print(testsubject)
    print('Новый контрагент: ', created_subjisn4)
    if created_subjisn == created_subjisn2 == created_subjisn3 == created_subjisn4:
        test_result = 'Ok'
    else:
        test_result = 'Fail'
writeresult(row, resp, created_subjisn, test_result)
row += 1

timeend = datetime.datetime.now()
print('Время, затраченное на тест:', timeend-timebeg)
result

Минимальный набор данных
Два паспорта


In [ ]:
result